In [ ]:
pip install opencv-python

In [1]:

import os
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Conv2D 
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2M, MobileNetV3Large
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay


DATA_PATH = './raw-img'
SEED=42
N_EPOCHS=10
BATCH_SIZE=32
INPUT_SIZE=(224, 224)

In [ ]:
keras.__version__

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.20, rescale=1./255, 
horizontal_flip=True, rotation_range = 10, width_shift_range = 0.05, height_shift_range = 0.05, brightness_range = (0.8, 1.2))


validation_datagen = ImageDataGenerator(validation_split=0.20, rescale=1./255)


train_datagen_flow = train_datagen.flow_from_directory(DATA_PATH, target_size=INPUT_SIZE,                                                   
batch_size=BATCH_SIZE, class_mode='sparse', subset='training', seed=SEED)

val_datagen_flow = validation_datagen.flow_from_directory(DATA_PATH, target_size=INPUT_SIZE,batch_size=BATCH_SIZE, class_mode='sparse', subset='validation', seed=SEED, shuffle = False)

In [ ]:
train_datagen_flow.class_indices

In [ ]:
val_datagen_flow.class_indices


In [ ]:
images, labels = next(train_datagen_flow)
images = images[:20]
labels = labels[:20]
cols = 5
rows = int(len(images)/cols) + 1
plt.figure(figsize=(30,20))
index = 1
for image, label in zip(images, labels):
    plt.subplot(rows, cols, index)
    plt.imshow(image)
    plt.title(f'Class: {int(label)}', fontsize = 15);
    index+=1
plt.show()

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape= INPUT_SIZE +(3,)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics =['acc'])


In [ ]:
model.summary()

In [ ]:
path_checkpoint = "model_checkpoint.keras"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
)

history = model.fit(
    train_datagen_flow,
    epochs=N_EPOCHS,
    validation_data=val_datagen_flow,
    callbacks=[es_callback, modelckpt_callback],
)

In [ ]:
def check_metrics(history, figure_name):

    train = history.history[f'{figure_name}']
    val = history.history[f'val_{figure_name}']

    epochs = range(1, len(train) + 1)
    plt.plot(epochs, train, label=f'Training {figure_name}', color='orange')
    plt.plot(epochs, val, label=f'Validation {figure_name}')
    plt.title(f'Training and Validation {figure_name}')
    plt.xlabel('Epochs')
    plt.ylabel(f'{figure_name}')
    plt.legend()
    plt.show()

check_metrics(history, 'loss')
check_metrics(history, 'acc')

In [ ]:
labels = ['cane', 'cavallo', 'elefante', 'farfalla','gallina','gatto','mucca', 'pecora','ragno', 'scoiattolo' ]

In [ ]:
model = load_model("model_checkpoint.keras")
model.evaluate(val_datagen_flow)

In [71]:
y_pred_indexes = []
y_true_indexes = []

In [ ]:
for i in range(len(val_datagen_flow)):
    x_test_batch, y_test_batch = val_datagen_flow[i]
    y_pred_batch = model.predict(x_test_batch)
    y_pred_index = np.argmax(y_pred_batch, axis = 1)
    y_pred_indexes.append(y_pred_index)
    y_true_indexes.append(y_test_batch)
    

In [ ]:
y_true_indexes = np.concatenate(y_true_indexes)
y_pred_indexes = np.concatenate(y_pred_indexes)

In [ ]:
cm = confusion_matrix(y_true_indexes, y_pred_indexes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

In [ ]:
print(classification_report(y_true_indexes, y_pred_indexes, target_names = labels))

In [ ]:
backbone = keras.applications.MobileNetV3Large(
    include_top=False,
    weights="imagenet",
    pooling='max',
    include_preprocessing=False,
)

In [16]:
backbone.trainable = False

In [17]:
model = Sequential()
model.add(backbone)
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics =['acc'])

In [ ]:
path_checkpoint = "model_checkpoint_mobile_net_v3.keras"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
)

history_mobile_net = model.fit(
    train_datagen_flow,
    epochs=N_EPOCHS,
    validation_data=val_datagen_flow,
    callbacks=[es_callback, modelckpt_callback],
)

In [ ]:
check_metrics(history_mobile_net, 'loss')
check_metrics(history_mobile_net, 'acc')

In [53]:
model = load_model("model_checkpoint_mobile_net_v3.keras")


In [ ]:
model.evaluate(val_datagen_flow) 

In [55]:
y_pred_indexes = []
y_true_indexes = []

In [ ]:
for i in range(len(val_datagen_flow)):
    x_test_batch, y_test_batch = val_datagen_flow[i]
    y_pred_batch = model.predict(x_test_batch)
    y_pred_index = np.argmax(y_pred_batch, axis = 1)
    y_pred_indexes.append(y_pred_index)
    y_true_indexes.append(y_test_batch)
    

In [59]:
y_pred_indexes = np.concatenate(y_pred_indexes)

In [62]:
y_true_indexes = np.concatenate(y_true_indexes)

In [ ]:
cm = confusion_matrix(y_true_indexes, y_pred_indexes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

In [ ]:
print(classification_report(y_true_indexes, y_pred_indexes, target_names = labels))

In [8]:
backbone = keras.applications.EfficientNetV2M(
    include_top=False,
    weights="imagenet",
    pooling='max',
    include_preprocessing=False,
)

In [9]:
backbone.trainable = False

In [10]:
model = Sequential()
model.add(backbone)
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics =['acc'])

In [ ]:
path_checkpoint = "model_checkpoint_efficent.keras"

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
)

history_mobile_net = model.fit(
    train_datagen_flow,
    epochs=N_EPOCHS,
    validation_data=val_datagen_flow,
    callbacks=[es_callback, modelckpt_callback],
)

In [ ]:
check_metrics(history_mobile_net, 'loss')
check_metrics(history_mobile_net, 'acc')

In [ ]:
model = load_model("model_checkpoint_mobile_net_v3.keras")
model.evaluate(val_datagen_flow) 

In [ ]:
y_pred_indexes = []
y_true_indexes = []

In [ ]:
for i in range(len(val_datagen_flow)):
    x_test_batch, y_test_batch = val_datagen_flow[i]
    y_pred_batch = model.predict(x_test_batch)
    y_pred_index = np.argmax(y_pred_batch, axis = 1)
    y_pred_indexes.append(y_pred_index)
    y_true_indexes.append(y_test_batch)

In [ ]:
y_pred_indexes = np.concatenate(y_pred_indexes)
y_true_indexes = np.concatenate(y_true_indexes)

In [ ]:
cm = confusion_matrix(y_true_indexes, y_pred_indexes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()